In [93]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        # print('-----------')
        # print(equivalence_class)
        for key_decision in equivalence_classes_dec.keys():
            # print(key_decision)
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(x)
            if(len(x)>total_records*0.0):
                y = 1-len(x) / class_size
                # if y<0.7:
                total_rule+=len(x)
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)

    rule_confidences = np.array(rule_confidences)
    # print(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.35:
    #     return (999,);


    avg_confidence = np.sum(rule_confidences) / count +0.6*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(42)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [94]:
def bestIndividual_KNN(hof,df):
    k = 2
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('../../data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [95]:
from sklearn.model_selection import StratifiedKFold
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('../../data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [96]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('../../data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [97]:
if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=30
    n_gen=30
    # read dataframe from csv
    df = pd.read_csv('../../data/an/anneal_train.csv')
    df['f33'] = df['f33'].astype(int)
    df['f34'] = df['f33'].astype(int)
    df['f35'] = df['f33'].astype(int)
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    # for col in feature_columns:
    #     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(0.7139003137226386,)

gen	nevals	avg     	min     
0  	30    	0.677352	0.587231
1  	21    	0.648022	0.557724
2  	21    	0.614057	0.557724
3  	24    	0.592334	0.536381
4  	11    	0.573505	0.504525
5  	18    	0.556784	0.453742
6  	22    	0.536616	0.453742
7  	23    	0.523383	0.453742
8  	11    	0.505804	0.453742
9  	24    	0.494182	0.453742
10 	25    	0.475156	0.42962 
11 	7     	0.458713	0.42962 
12 	16    	0.454431	0.414159
13 	17    	0.456156	0.414159
14 	20    	0.435267	0.414159
15 	17    	0.450662	0.40166 
16 	19    	0.429923	0.386356
17 	23    	0.438209	0.386356
18 	15    	0.409229	0.386356
19 	14    	0.40458 	0.386356
20 	23    	0.415061	0.319787
21 	15    	0.386839	0.319787
22 	17    	0.369029	0.319787
23 	20    	0.355952	0.319787
24 	20    	0.326873	0.292058
25 	22    	0.332849	0.292058
26 	15    	0.320849	0.292058
27 	14    	0.317582	0.292058
28 	15    	0.314545	0.292058
29 	16    	0.309428	0.292058
30 	20    	0.294666	0.280768
Top 1 solution: [0, 

In [98]:
accuracy_KNN,individual = bestIndividual_KNN(hof,df)
print("-----------KNN---------------")
print(accuracy_KNN)

-----------KNN---------------
0.7917353308364544


In [99]:
print('------------特征-----------------')
print(individual)
print(len(individual))

------------特征-----------------
['f2', 'f4', 'f14', 'f27', 'f28', 'f29', 'f31', 'f33', 'f34', 'f35', 'f37', 'f38']
12


In [100]:
accuracy_SVM,individual = bestIndividual_SVM(hof,df)
print("-----------SVM---------------")
print(accuracy_SVM)

-----------SVM---------------
0.7983645443196006


In [101]:
print("-----------DT---------------")
accuracy_DT = bestIndividual_DT(hof,df)
print(accuracy_DT)

-----------DT---------------
0.8184269662921346
